---
title: C++ const重载不起作用的情况
tags: 小书匠,c++,const,overwrite
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ const重载不起作用的情况

## 例1

In [47]:
%%file test.cpp
#include<iostream>  

void fun(const int i)  
{   
    std::cout << "fun(const int) called \n";  
}  

void fun(int i)  
{  
    std::cout << "fun(int ) called \n" ;  
}  

int main()  
{  
    int a = 10
    fun(a);  
    
    const int b = 10;  
    fun(b);  
    
    return 0;  
}

Overwriting test.cpp


In [48]:
!g++ test.cpp

test.cpp:8:6: error: redefinition of 'fun'
void fun(int i)  
     ^
test.cpp:3:6: note: previous definition is here
void fun(const int i)  
     ^
test.cpp:15:15: error: expected ';' at end of declaration
    int a = 10
              ^
              ;
2 errors generated.


发现报错了，因为 `func(const int i)` 和 `func(int i)` 时重定义的，因为编译器将其看作是同一个函数。

为什么会这样？明明一个是 const，一个不是 const？

这是因为 `i` 采用的是值传递，函数内部的 `i` 本身就不会改变函数外边的 `a` 和 `b`，因此这里使用 const 是无效的。

## 例2

再来看下面两个例子

In [55]:
%%file test.cpp
#include<iostream>  
using namespace std;  
   
void fun(char *a)  
{  
  cout << "non-const fun()\n";
}  
   
void fun(const char *a)  
{  
  cout << "const fun()\n";
}  
   
int main()  
{  
    char *ptr1 = "hello world";
    const char *ptr2 = "hello world";  
    fun(ptr1);
    fun(ptr2);  
    return 0;  
}

Overwriting test.cpp


In [56]:
!g++ test.cpp && ./a.out

non-const fun()
const fun()


例2的情况是正确的。因为 `const char* a` 表示的是指向常量的指针。

## 例3

In [62]:
%%file test.cpp
#include<iostream>  
using namespace std;  
   
void fun(char* a)  
{  
  cout << "non-const fun()\n" << a;  
}  
   
void fun(char* const a)  
{  
  cout << "const fun()\n" << a;  
}  
   
int main()  
{  
    char *ptr1 = "hello world";
    const char *ptr2 = "hello world";  
    fun(ptr1);
    fun(ptr2);  
    return 0;  
}

Overwriting test.cpp


In [63]:
!g++ test.cpp

test.cpp:9:6: error: redefinition of 'fun'
void fun(char* const a)  
     ^
test.cpp:4:6: note: previous definition is here
void fun(char* a)  
     ^
test.cpp:16:18: warning: conversion from string literal to 'char *' is deprecated [-Wc++11-compat-deprecated-writable-strings]
    char *ptr1 = "hello world";
                 ^
1 warning and 1 error generated.


情况2又报错了，这个例1实际上是同一个问题。因为 `const char* a` 表示的是常量指针，表示指针本身是个常量，它指向的内容是可以边的。而由于采用的是值传递，因此函数内部的 `a` 不会影响到函数外面的 `ptr1`、`ptr2`，因此会报错

# References

- [C++中const用于函数重载 - 青儿哥哥 - 博客园](https://www.cnblogs.com/qingergege/p/7609533.html)